In [151]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from dataset import transformed_dataset
from model import NeuralNetwork
from torch.utils.data import random_split

In [152]:
model = NeuralNetwork()
parameters = [[name, p] for name, p in model.named_parameters()]
print(parameters[0][1].data)

tensor([[-0.0304,  0.0856,  0.0710,  ..., -0.0663, -0.0106,  0.0038],
        [ 0.0145,  0.0768,  0.1124,  ...,  0.1012, -0.0841, -0.0474],
        [-0.0124, -0.0956,  0.0751,  ..., -0.0520, -0.0757,  0.0200],
        ...,
        [-0.0232, -0.0909, -0.0861,  ...,  0.0979,  0.1191,  0.0086],
        [-0.0970,  0.0275, -0.0852,  ...,  0.0692, -0.0305,  0.0668],
        [ 0.0373, -0.0755, -0.0775,  ..., -0.0107,  0.1174, -0.0726]])


In [153]:
names = [name for name, p in parameters]
names

['linear_relu_stack.0.weight',
 'linear_relu_stack.0.bias',
 'linear_relu_stack.2.weight',
 'linear_relu_stack.2.bias',
 'linear_relu_stack.4.weight',
 'linear_relu_stack.4.bias',
 'linear_relu_stack.6.weight',
 'linear_relu_stack.6.bias']

In [154]:
def create_random_tensor(*args):
    array = np.random.rand(*args) / 10
    # TODO: add negative numbers to random arrays
    # for _ in range(array.size * 0.5):
    return array

params_sets = []
for _ in range(100):
    tmp = []
    tmp.append(np.random.rand(64, 64) / 10)
    tmp.append(np.random.rand(64) / 10)
    tmp.append(np.random.rand(64, 64) / 10)
    tmp.append(np.random.rand(64) / 10)    
    tmp.append(np.random.rand(64, 64) / 10)
    tmp.append(np.random.rand(64) / 10)
    tmp.append(np.random.rand(10, 64) / 10)
    tmp.append(np.random.rand(10) / 10)
    params_sets.append(tmp)

In [155]:
def load_params_to_model(params):
    pretrained_dict = model.state_dict()
    counter = 0

    for key in pretrained_dict.keys():
        pretrained_dict[key] = torch.from_numpy(params[counter])
        counter += 1

    model.load_state_dict(pretrained_dict)
    parameters = [[name, p] for name, p in model.named_parameters()]
    print(parameters[0][1].data)

In [156]:
def eval_loop(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    return test_loss

In [157]:
dataset = random_split(
    transformed_dataset, (898, 899)
)  # the tuple elements have to sum up to 1797
train_dataset = dataset[0]
test_dataset = dataset[1]

batch_size = 40

train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)

def evaluate(params):
    load_params_to_model(params)
    score = eval_loop(train_dataloader, model, nn.CrossEntropyLoss())
    return score

In [158]:
def evaluate_all(param_list):
    scores = []
    for elem in param_list:
        scores.append(evaluate(elem))
    return scores

In [165]:
tmp = zip(params_sets, evaluate_all(params_sets))
new_list = sorted(list(tmp), key=lambda x: x[1])

tensor([[0.0329, 0.0683, 0.0273,  ..., 0.0346, 0.0715, 0.0218],
        [0.0880, 0.0634, 0.0294,  ..., 0.0475, 0.0500, 0.0068],
        [0.0784, 0.0743, 0.0111,  ..., 0.0440, 0.0637, 0.0047],
        ...,
        [0.0341, 0.0798, 0.0833,  ..., 0.0180, 0.0202, 0.0188],
        [0.0638, 0.0845, 0.0726,  ..., 0.0981, 0.0201, 0.0697],
        [0.0896, 0.0740, 0.0770,  ..., 0.0994, 0.0337, 0.0194]])
tensor([[0.0049, 0.0813, 0.0424,  ..., 0.0408, 0.0865, 0.0700],
        [0.0001, 0.0451, 0.0210,  ..., 0.0569, 0.0979, 0.0858],
        [0.0180, 0.0396, 0.0408,  ..., 0.0466, 0.0422, 0.0845],
        ...,
        [0.0920, 0.0789, 0.0310,  ..., 0.0697, 0.0462, 0.0987],
        [0.0335, 0.0514, 0.0243,  ..., 0.0958, 0.0456, 0.0425],
        [0.0078, 0.0223, 0.0513,  ..., 0.0727, 0.0516, 0.0575]])
tensor([[0.0948, 0.0992, 0.0529,  ..., 0.0545, 0.0923, 0.0760],
        [0.0982, 0.0126, 0.0095,  ..., 0.0999, 0.0761, 0.0202],
        [0.0208, 0.0020, 0.0889,  ..., 0.0794, 0.0327, 0.0321],
        ...,

In [181]:
newer_list = [x[1] / -100 for x in new_list]

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

softmax(newer_list)

array([0.01398264, 0.01397554, 0.01341466, 0.01331965, 0.01309148,
       0.0130545 , 0.01301543, 0.01289968, 0.01289601, 0.01288379,
       0.01265525, 0.01264974, 0.01259548, 0.01239475, 0.01228438,
       0.01218741, 0.01214757, 0.01192045, 0.01190092, 0.01189689,
       0.01180741, 0.01179978, 0.011676  , 0.01165727, 0.01163813,
       0.01163357, 0.01140327, 0.01134144, 0.01124975, 0.01118834,
       0.01114094, 0.01114025, 0.01105463, 0.01102722, 0.01089448,
       0.01086082, 0.01083389, 0.01076638, 0.0107274 , 0.01071671,
       0.01047585, 0.01042278, 0.01041264, 0.01040924, 0.01036043,
       0.01033825, 0.01030233, 0.0102989 , 0.01027677, 0.01025555,
       0.01019224, 0.01016715, 0.01000574, 0.00997055, 0.00995226,
       0.0098682 , 0.00986538, 0.00968939, 0.00963676, 0.00946076,
       0.00944257, 0.00931504, 0.0091723 , 0.00912615, 0.00910322,
       0.00908985, 0.00903679, 0.00889389, 0.00884944, 0.00882902,
       0.00875355, 0.00862304, 0.00854508, 0.00853583, 0.00850

In [1]:
import numpy as np

# input array
x = np.array([[ 1., 2., 3.], [ 4., 5., 6.], [ 7., 8., 9.]])

# random boolean mask for which values will be changed
mask = np.invert(np.random.randint(0,2,size=x.shape).astype(bool))

# random matrix the same shape of your data
r = (np.random.rand(*x.shape) - 0.5) * 0.4
print(x)

# use your mask to replace values in your input array
x[mask] = -r[mask]
print(x)

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
[[ 0.02841362  0.10933008  3.        ]
 [ 4.          5.          6.        ]
 [-0.13447874 -0.13602066  9.        ]]


In [11]:
[1, 2] * 2

[1, 2, 1, 2]